In [190]:
import random
import numpy as np

# Initial data given in problem
tasks = [5, 8, 4, 7, 6, 3, 9] 
facilities = [24, 30, 28]  
cost_matrix = [
    [10, 12, 9],
    [15, 14, 16],
    [8, 9, 7],
    [12, 10, 13],
    [14, 13, 12],
    [9, 8, 10],
    [11, 12, 13]
]

POP_SIZE = 6
CROSSOVER_RATE = 0.8
MUTATION_RATE = 0.2
GENERATIONS = 50

In [191]:
# Generating population
def generate_population():
    return [random.choices([1, 2, 3], k=len(tasks)) for i in range(POP_SIZE)]


generate_population()

[[2, 1, 1, 3, 1, 1, 3],
 [2, 3, 3, 3, 3, 1, 2],
 [2, 3, 3, 2, 3, 1, 3],
 [3, 2, 3, 1, 3, 3, 1],
 [3, 3, 1, 1, 2, 1, 3],
 [1, 2, 2, 1, 3, 3, 2]]

In [192]:
# Fitness function
def fitness(cur):
    total_cost = 0
    loads = [0, 0, 0] # each index express a facility 
    
    for i, facility in enumerate(cur):
        facility_index = facility - 1
        loads[facility_index] += tasks[i]
        total_cost += tasks[i] * cost_matrix[i][facility_index]
    
     # Applying penalty for overcapacity
    for i, load in enumerate(loads):
        if load > facilities[i]:
            total_cost += (load - facilities[i]) * 100  # each extra hour costs 100 rs
    
    return -total_cost  
    

In [193]:
# Selection (Roulette Wheel)
def select(population):
    fitness_values = [fitness(chrom) for chrom in population]
    total_fitness = sum(fitness_values)
    probabilities = [f / total_fitness for f in fitness_values]
    return population[np.random.choice(len(population), p=probabilities)]

# Crossover (One-Point)
def crossover(parent1, parent2):
    if random.random() < CROSSOVER_RATE:
        point = random.randint(1, len(tasks) - 1)
        return parent1[:point] + parent2[point:], parent2[:point] + parent1[point:]
    return parent1, parent2

# Mutation (Swap Mutation)
def mutate(cur): 
    if random.random() < MUTATION_RATE:
        idx1, idx2 = random.sample(range(len(tasks)), 2)
        cur[idx1], cur[idx2] = cur[idx2], cur[idx1]
    return cur


In [297]:
# Genetic Algorithm
def genetic_algorithm():
    population = generate_population()
    
    for generation in range(GENERATIONS):
        new_population = []
        
        for _ in range(POP_SIZE // 2):
            parent1, parent2 = select(population), select(population)
            child1, child2 = crossover(parent1, parent2)
            new_population.extend([mutate(child1), mutate(child2)])
        
        population = new_population

    best_solution = min(population, key=fitness)
    best_cost = -fitness(best_solution)
    return best_solution, best_cost


best_assignment, best_cost = genetic_algorithm()
print("Best Task Allocation:", best_assignment)
print("Minimum Production Cost:", best_cost)

Best Task Allocation: [2, 1, 2, 3, 1, 2, 2]
Minimum Production Cost: 523
